In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import copy
import json
from glob import glob
import time
import os
from os.path import join
import pickle as pkl

import sys
sys.path.append('../../')

from data.processing import get_data

from utils.fitting.loss import Loss_Calculator
from viz import plot_backtest_seir

# Jul

In [ ]:
run_name = "1594480618.481472"
output_folder = '../../misc/reports/{}'.format(run_name)

In [ ]:
filename = f'{output_folder}/output-{run_name}.csv'

In [ ]:
plot_backtest_seir(gt_data_source='athena', preds_source='filename', fname_format='old_output', 
                   filename=filename, which_forecast=50, truncate_plotting_range=True,
                   separate_compartments=False)

In [ ]:
plot_backtest_seir(gt_data_source='athena', preds_source='filename', fname_format='old_output', 
                   filename=filename, which_forecast=50, truncate_plotting_range=True,
                   separate_compartments=True)

# Aug

In [ ]:
filename = '../../misc/oncall/08_2020_aug/nayana/seihrd.csv'

In [ ]:
plot_backtest_seir(gt_data_source='athena', preds_source='filename', fname_format='new_deciles', 
                   filename=filename, which_forecast=80, truncate_plotting_range=True,
                   separate_compartments=False)

# Sept

In [ ]:
output_folder = '../../misc/reports/{}'.format("2020_0908_090609")
with open(join(output_folder, 'predictions_dict.pkl'), 'rb') as f:
    predictions_dict = pkl.load(f)

In [ ]:
plot_backtest_seir(gt_data_source='athena', preds_source='pickle', predictions_dict=predictions_dict, 
                   which_forecast=80, truncate_plotting_range=True,
                   separate_compartments=False)

In [ ]:
plot_backtest_seir(gt_data_source='athena', preds_source='pickle', predictions_dict=predictions_dict, 
                   which_forecast=80, truncate_plotting_range=True,
                   separate_compartments=True)

In [ ]:
lc = Loss_Calculator()
forecast_errors_dict = {}
df_train = predictions_dict['m2']['df_train']

for key, df_prediction in predictions_dict['m2']['forecasts'].items():
    if key in ['best', 50, 80]:
        df_prediction = copy.copy(df_prediction.loc[(df_prediction['date'] > df_train.iloc[-1, :]['date']) & 
                                                    (df_prediction['date'] <= df_true.iloc[-1, :]['date'])])
        df_true = df_true.loc[df_true['date'] > df_train.iloc[-1, :]['date']]
        df_prediction.reset_index(inplace=True, drop=True)
        df_true.reset_index(inplace=True, drop=True)
        ld = lc.calc_loss_dict(df_prediction, df_true, method='mape')
        ld = {key : round(value, 2) for key, value in ld.items()}
        forecast_errors_dict[key] = ld

In [ ]:
pd.DataFrame.from_dict(forecast_errors_dict)

In [ ]:
lc = Loss_Calculator()
forecast_errors_dict = {}
df_train = predictions_dict['m2']['df_train']

for key, df_prediction in predictions_dict['m2']['forecasts'].items():
    if key in [80]:
        df_prediction = copy.copy(df_prediction.loc[(df_prediction['date'] > df_train.iloc[-1, :]['date']) & 
                                                    (df_prediction['date'] <= df_true.iloc[-1, :]['date'])])
        df_true = copy.copy(df_true.loc[df_true['date'] > df_train.iloc[-1, :]['date']])
        df_prediction.reset_index(inplace=True, drop=True)
        df_true.reset_index(inplace=True, drop=True)
        week_indices = np.concatenate((np.arange(0, len(df_true), 7), [len(df_true)]))
        for i in range(len(week_indices)-1):
            df_prediction_slice = df_prediction.loc[week_indices[i]:week_indices[i+1]-1, :]
            df_true_slice = df_true.loc[week_indices[i]:week_indices[i+1]-1, :]
        
            ld = lc.calc_loss_dict(df_prediction_slice, df_true_slice, method='mape')
            
            ld = {key : round(value, 2) for key, value in ld.items()}
            forecast_errors_dict[f'week {i+1}'] = ld
            
        ld = lc.calc_loss_dict(df_prediction, df_true, method='mape')
            
        ld = {key : round(value, 2) for key, value in ld.items()}
        forecast_errors_dict['total'] = ld

In [ ]:
df = pd.DataFrame.from_dict(forecast_errors_dict)
df.index = ['active', 'recovered', 'deceased', 'total']
df

In [ ]:
predictions_dict['m2']['forecasts'][80]